In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator

from indra_db_lite.api import get_entrez_pmids_for_hgnc
from indra_db_lite.api import get_entrez_pmids_for_uniprot
from indra_db_lite.api import get_plaintexts_for_text_ref_ids
from indra_db_lite.api import get_text_ref_ids_for_agent_text
from indra_db_lite.api import get_text_ref_ids_for_pmids


from adeft_indra.grounding import AdeftGrounder
from adeft_indra.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename

In [2]:
def get_text_ref_ids_for_entity(ns, id_):
    if ns == 'HGNC':
        pmids = get_entrez_pmids_for_hgnc(id_)
    elif ns == 'UP':
        pmids = get_entrez_pmids_for_uniprot(id_)
    return list(get_text_ref_ids_for_pmids(pmids).values())

In [3]:
adeft_grounder = AdeftGrounder()

In [4]:
shortforms = ['HCS']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../../', 'results', model_name))

In [5]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    text_ref_ids = get_text_ref_ids_for_agent_text(shortform)
    content = get_plaintexts_for_text_ref_ids(text_ref_ids, contains=shortforms)
    text_dict = content.flatten()
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [6]:
list(zip(longforms, counts))

[('holocarboxylase synthetase', 34),
 ('high content screening', 21),
 ('hajdu cheney syndrome', 11),
 ('human corneal stromal', 7),
 ('heliospheric current sheet', 7),
 ('healthy control subjects', 5),
 ('hunter community study', 5),
 ('hollow carbon sphere', 4),
 ('high contractile state', 4),
 ('histological combined score', 4),
 ('hereditary cancer solution', 4),
 ('hmsc cell sheet', 4),
 ('homocitrate synthase', 4),
 ('huachansu', 4),
 ('hydrocortisone', 4),
 ('human infection challenge study', 4),
 ('high concentrate diet', 4),
 ('hmg coa synthase', 3),
 ('chitosan hydrochloride', 3)]

In [6]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

In [7]:
result = [grounding_map, names, pos_labels]

In [8]:
result

[{'chitosan hydrochloride': 'ungrounded',
  'hajdu cheney syndrome': 'MESH:D031845',
  'healthy control subjects': 'ungrounded',
  'heliospheric current sheet': 'ungrounded',
  'hereditary cancer solution': 'ungrounded',
  'high concentrate diet': 'ungrounded',
  'high content screening': 'EFO:0007550',
  'high contractile state': 'ungrounded',
  'histological combined score': 'ungrounded',
  'hmg coa synthase': 'HGNC:5007',
  'hmsc cell sheet': 'ungrounded',
  'hollow carbon sphere': 'ungrounded',
  'holocarboxylase synthetase': 'HGNC:4976',
  'homocitrate synthase': 'IP:IPR011872',
  'huachansu': 'MESH:C545813',
  'human corneal stromal': 'NCIT:NCIT:C12699',
  'human infection challenge study': 'ungrounded',
  'hunter community study': 'ungrounded',
  'hydrocortisone': 'CHEBI:CHEBI:17650'},
 {'MESH:D031845': 'Hajdu-Cheney Syndrome',
  'EFO:0007550': 'high content screen',
  'HGNC:5007': 'HMGCS1',
  'HGNC:4976': 'HLCS',
  'IP:IPR011872': 'Homocitrate synthase, fungi/archaea',
  'MESH:

In [9]:
grounding_map, names, pos_labels = [{'chitosan hydrochloride': 'ungrounded',
  'hajdu cheney syndrome': 'MESH:D031845',
  'healthy control subjects': 'ungrounded',
  'heliospheric current sheet': 'ungrounded',
  'hereditary cancer solution': 'ungrounded',
  'high concentrate diet': 'ungrounded',
  'high content screening': 'EFO:0007550',
  'high contractile state': 'ungrounded',
  'histological combined score': 'ungrounded',
  'hmg coa synthase': 'HGNC:5007',
  'hmsc cell sheet': 'ungrounded',
  'hollow carbon sphere': 'ungrounded',
  'holocarboxylase synthetase': 'HGNC:4976',
  'homocitrate synthase': 'IP:IPR011872',
  'huachansu': 'MESH:C545813',
  'human corneal stromal': 'NCIT:NCIT:C12699',
  'human infection challenge study': 'ungrounded',
  'hunter community study': 'ungrounded',
  'hydrocortisone': 'CHEBI:CHEBI:17650'},
 {'MESH:D031845': 'Hajdu-Cheney Syndrome',
  'EFO:0007550': 'high content screen',
  'HGNC:5007': 'HMGCS1',
  'HGNC:4976': 'HLCS',
  'IP:IPR011872': 'Homocitrate synthase, fungi/archaea',
  'MESH:C545813': 'huachansu',
  'NCIT:NCIT:C12699': 'Corneal Stroma',
  'CHEBI:CHEBI:17650': 'cortisol'},
 ['HGNC:4976']]

In [10]:
excluded_longforms = []

In [11]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [12]:
additional_entities = {
    'HGNC:4976': ['HLCS', ['HCS', 'HLCS']],
    'HGNC:19986': ['CYCS', ['HCS']],
    'HGNC:5007': ['HMGCS1', ['HCS']],
}

In [13]:
unambiguous_agent_texts = {}

In [14]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts(
    (text, text_ref_id) for text_ref_id, text in all_texts.items()
)
agent_text_text_ref_id_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_text_ref_id_map[label].append(id_)

entity_text_ref_id_map = {
    entity: set(
        get_text_ref_ids_for_entity(*entity.split(':', maxsplit=1))
    )
    for entity in additional_entities
}

In [15]:
intersection1 = []
for entity1, trids1 in entity_text_ref_id_map.items():
    for entity2, trids2 in entity_text_ref_id_map.items():
        intersection1.append((entity1, entity2, len(trids1 & trids2)))

In [16]:
intersection2 = []
for entity1, trids1 in agent_text_text_ref_id_map.items():
    for entity2, pmids2 in entity_text_ref_id_map.items():
        intersection2.append((entity1, entity2, len(set(trids1) & trids2)))

In [17]:
intersection1

[('HGNC:4976', 'HGNC:4976', 59),
 ('HGNC:4976', 'HGNC:19986', 9),
 ('HGNC:4976', 'HGNC:5007', 8),
 ('HGNC:19986', 'HGNC:4976', 9),
 ('HGNC:19986', 'HGNC:19986', 185),
 ('HGNC:19986', 'HGNC:5007', 9),
 ('HGNC:5007', 'HGNC:4976', 8),
 ('HGNC:5007', 'HGNC:19986', 9),
 ('HGNC:5007', 'HGNC:5007', 38)]

In [19]:
intersection2

[('ungrounded', 'HGNC:4976', 0),
 ('ungrounded', 'HGNC:19986', 0),
 ('ungrounded', 'HGNC:5007', 0),
 ('HGNC:4976', 'HGNC:4976', 0),
 ('HGNC:4976', 'HGNC:19986', 0),
 ('HGNC:4976', 'HGNC:5007', 0),
 ('IP:IPR011872', 'HGNC:4976', 0),
 ('IP:IPR011872', 'HGNC:19986', 0),
 ('IP:IPR011872', 'HGNC:5007', 0),
 ('EFO:0007550', 'HGNC:4976', 0),
 ('EFO:0007550', 'HGNC:19986', 0),
 ('EFO:0007550', 'HGNC:5007', 0),
 ('NCIT:NCIT:C12699', 'HGNC:4976', 0),
 ('NCIT:NCIT:C12699', 'HGNC:19986', 0),
 ('NCIT:NCIT:C12699', 'HGNC:5007', 0),
 ('MESH:D031845', 'HGNC:4976', 0),
 ('MESH:D031845', 'HGNC:19986', 0),
 ('MESH:D031845', 'HGNC:5007', 0),
 ('MESH:C545813', 'HGNC:4976', 0),
 ('MESH:C545813', 'HGNC:19986', 0),
 ('MESH:C545813', 'HGNC:5007', 0),
 ('HGNC:5007', 'HGNC:4976', 0),
 ('HGNC:5007', 'HGNC:19986', 0),
 ('HGNC:5007', 'HGNC:5007', 0),
 ('CHEBI:CHEBI:17650', 'HGNC:4976', 0),
 ('CHEBI:CHEBI:17650', 'HGNC:19986', 0),
 ('CHEBI:CHEBI:17650', 'HGNC:5007', 0)]

In [20]:
all_used_trids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_trids = set()
    for agent_text in agent_texts[1]:
        trids = set(get_text_ref_ids_for_agent_text(agent_text))
        new_trids = list(trids - all_texts.keys() - used_trids)
        content = get_plaintexts_for_text_ref_ids(new_trids, contains=agent_texts[1])
        text_dict = content.flatten()
        corpus.extend(
            [
                (text, entity, trid) for trid, text in text_dict.items() if len(text) >= 5
            ]
        )
        used_trids.update(new_trids)
    all_used_trids.update(used_trids)
        
for entity, trids in entity_text_ref_id_map.items():
    new_trids = list(set(trids) - all_texts.keys() - all_used_trids)
    _, contains = additional_entities[entity]
    content = get_plaintexts_for_text_ref_ids(new_trids, contains=contains)
    text_dict = content.flatten()
    corpus.extend(
        [
            (text, entity, trid) for trid, text in text_dict.items() if len(text) >= 5
        ]
    )

In [21]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [22]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2021-10-07 19:58:36] /adeft/Py/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2021-10-07 19:58:40] /adeft/Py/adeft/adeft/modeling/classify.py - Best f1 score of 0.9436047171219585 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [23]:
classifier.stats

{'label_distribution': {'ungrounded': 19,
  'HGNC:4976': 53,
  'IP:IPR011872': 3,
  'EFO:0007550': 14,
  'NCIT:NCIT:C12699': 6,
  'MESH:D031845': 5,
  'MESH:C545813': 3,
  'HGNC:5007': 2,
  'CHEBI:CHEBI:17650': 3,
  'HGNC:19986': 3},
 'f1': {'mean': 0.943605, 'std': 0.037233},
 'precision': {'mean': 0.928934, 'std': 0.069406},
 'recall': {'mean': 0.966667, 'std': 0.066667},
 'CHEBI:CHEBI:17650': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'EFO:0007550': {'f1': {'mean': 0.791429, 'std': 0.132588},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 0.675714, 'std': 0.191429}},
 'HGNC:19986': {'f1': {'mean': 0.4, 'std': 0.489898},
  'pr': {'mean': 0.4, 'std': 0.489898},
  'rc': {'mean': 0.4, 'std': 0.489898}},
 'HGNC:4976': {'f1': {'mean': 0.947254, 'std': 0.038516},
  'pr': {'mean': 0.98, 'std': 0.04},
  'rc': {'mean': 0.922294, 'std': 0.078659}},
 'HGNC:5007': {'f1': {'mean': 0.4, 'std': 0.489898},
  'pr': {'mean': 0.4, 'std

In [24]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [25]:
disamb.dump(model_name, results_path)

In [26]:
print(disamb.info())

Disambiguation model for HCS

Produces the disambiguations:
	CYCS*	HGNC:19986
	Corneal Stroma	NCIT:NCIT:C12699
	HLCS*	HGNC:4976
	HMGCS1*	HGNC:5007
	Hajdu-Cheney Syndrome	MESH:D031845
	Homocitrate synthase, fungi/archaea	IP:IPR011872
	cortisol	CHEBI:CHEBI:17650
	high content screen	EFO:0007550
	huachansu	MESH:C545813

Class level metrics:
--------------------
Grounding                          	Count	F1     
                               HLCS*	53	0.94725
                         Ungrounded	19	0.79254
                high content screen	14	0.79143
                     Corneal Stroma	 6	0.93333
              Hajdu-Cheney Syndrome	 5	    1.0
Homocitrate synthase, fungi/archaea	 3	    0.0
                          huachansu	 3	    0.0
                           cortisol	 3	    0.0
                               CYCS*	 3	    0.4
                             HMGCS1*	 2	    0.4

Global Metrics:
-----------------
	F1 score:	0.9436
	Precision:	0.92893
	Recall:		0.96667

* Positive labels
See Do

In [27]:
model_to_s3(disamb)

In [28]:
from adeft.disambiguate import load_disambiguator

In [29]:
disamb = load_disambiguator("BAL")

In [30]:
disamb

/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1495: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is 

In [29]:
print(_28.info())

Disambiguation model for BAL

Produces the disambiguations:
	Bronchoalveolar Lavage	MESH:D018893
	CEL*	HGNC:1848
	Liver, Artificial	MESH:D019164
	benzaldehyde lyase*	MESH:C059416
	betaine aldehyde*	CHEBI:CHEBI:15710
	dimercaprol*	CHEBI:CHEBI:64198

Class level metrics:
--------------------
Grounding             	Count	F1     
Bronchoalveolar Lavage	1259	 0.9929
                   CEL*	  36	    1.0
     Liver, Artificial	  18	0.83619
            Ungrounded	  17	   0.65
           dimercaprol*	   8	    0.4
    benzaldehyde lyase*	   3	    0.2
      betaine aldehyde*	   2	    0.2

Global Metrics:
-----------------
	F1 score:	0.90773
	Precision:	1.0
	Recall:		0.83293

* Positive labels
See Docstring for explanation



/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1495: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is 